# 本页代码用于数据分析
## 1、获取数据
## 2、相关性分析：使用pandas自带的相关性分析，考虑时滞，结论：筛选数据项有效性
## 3、非线性回归：使用神经网络回归，结论：研究预测模型可行性
## 4、基础统计与概率分布：直接统计数据获取概率分布于条件概率
## 5、动力系统相关：通过Koopman算子获取特征值，判断收敛稳定性

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

# 获取数据

In [3]:
tankid='3'
data=pd.read_csv('./Tank'+tankid+'/data_new.csv')
print(data.columns)
n,m=data.shape

Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '2号剩余泵瞬时流量', '2号生物反应池瞬时流量', '6-3号生物池NH3_1', '6-3号生物池NH3_2',
       '6-3号生物池厌氧段MLSS', '6-3号生物池好氧段MLSS', '6-3号生物池好氧段NO3', '6-3号生物池缺氧段MLSS',
       '6-3好氧段曝气量1', '6-3好氧段曝气量2', '6-3好氧段曝气量3', '6-3好氧段曝气量4', '6-3生物池DO1',
       '6-3生物池DO2', '6-3生物池DO3', '6-3生物池DO4', '7-1外回流流量2', '内回流流量2'],
      dtype='object')


# 相关性分析

In [4]:
# lag
lags=[1,30,60,90,120]
#lags=[i for i in range(0,2000,50)]
delt=10
n = int(n/3)
rc = {'font.sans-serif': 'SimHei',
        'axes.unicode_minus': False}
re=['pearson']#'kendall',,'spearman'
table=[]
for lag in lags:
    for r in re:
        tem=[]
        for tankid in [str(i+1) for i in range(7)]:
            data=pd.read_csv('./Tank'+tankid+'/data_new.csv')
            print(data.columns)
            n,m=data.shape

            ind=['6-'+tankid+'好氧段曝气量1', '6-'+tankid+'好氧段曝气量2', '6-'+tankid+'好氧段曝气量3', '6-'+tankid+'好氧段曝气量4']
            ind_lag=['6-'+tankid+'生物池DO1', '6-'+tankid+'生物池DO2', '6-'+tankid+'生物池DO3', '6-'+tankid+'生物池DO4']
            
            temdata=data.copy()
            temdata[ind_lag].iloc[0:n-lag-delt,:]=temdata[ind_lag].iloc[lag+delt:n,:]
            temdata=temdata.iloc[:n-lag-delt,2:]
        
            mat=temdata[ind+ind_lag].corr(method=r)
            val = mat.loc[ind,['6-'+tankid+'生物池DO1','6-'+tankid+'生物池DO2']].max().max()
            tem.append(val)
        table.append(tem)
pd.DataFrame(table).to_csv('./corr.csv')

Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '1号剩余泵瞬时流量', '1号生物反应池瞬时流量', '6-1号生物池NH3_1', '6-1号生物池NH3_2',
       '6-1号生物池厌氧段MLSS', '6-1号生物池好氧段MLSS', '6-1号生物池好氧段NO3', '6-1号生物池缺氧段MLSS',
       '6-1好氧段曝气量1', '6-1好氧段曝气量2', '6-1好氧段曝气量3', '6-1好氧段曝气量4', '6-1生物池DO1',
       '6-1生物池DO2', '6-1生物池DO3', '6-1生物池DO4', '7-1外回流流量1', '内回流流量1'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '1号剩余泵瞬时流量', '1号生物反应池瞬时流量', '6-2号生物池NH3_1', '6-2号生物池NH3_2',
       '6-2号生物池厌氧段MLSS', '6-2号生物池好氧段MLSS', '6-2号生物池好氧段NO3', '6-2号生物池缺氧段MLSS',
       '6-2好氧段曝气量1', '6-2好氧段曝气量2', '6-2好氧段曝气量3', '6-2好氧段曝气量4', '6-2生物池DO1',
       '6-2生物池DO2', '6-2生物池DO3', '6-2生物池DO4', '7-1外回流流量1', '内回流流量1'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '2号剩余泵瞬时流量', '2号生物反应池瞬时流量', '6-3号生物池NH3_1', '6-3号生物池NH3_2',
       '6-3号生物池厌氧段MLSS', '6-3号生物池好氧段MLSS', '6-3号生

c:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-4-3e90bb9b6af8>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temdata[ind_lag].iloc[0:n-lag-delt,:]=temdata[ind_lag].iloc[lag+delt:n,:]


Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '2号剩余泵瞬时流量', '2号生物反应池瞬时流量', '6-4号生物池NH3_1', '6-4号生物池NH3_2',
       '6-4号生物池厌氧段MLSS', '6-4号生物池好氧段MLSS', '6-4号生物池好氧段NO3', '6-4号生物池缺氧段MLSS',
       '6-4好氧段曝气量1', '6-4好氧段曝气量2', '6-4好氧段曝气量3', '6-4好氧段曝气量4', '6-4生物池DO1',
       '6-4生物池DO2', '6-4生物池DO3', '6-4生物池DO4', '7-1外回流流量2', '内回流流量2'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '3号剩余泵瞬时流量', '3号生物反应池瞬时流量', '6-5号生物池NH3_1', '6-5号生物池NH3_2',
       '6-5号生物池厌氧段MLSS', '6-5号生物池好氧段MLSS', '6-5号生物池好氧段NO3', '6-5号生物池缺氧段MLSS',
       '6-5好氧段曝气量1', '6-5好氧段曝气量2', '6-5好氧段曝气量3', '6-5好氧段曝气量4', '6-5生物池DO1',
       '6-5生物池DO2', '6-5生物池DO3', '6-5生物池DO4', '7-1外回流流量3', '内回流流量3'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 0.1', '进水COD', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '3号剩余泵瞬时流量', '3号生物反应池瞬时流量', '6-6号生物池NH3_1', '6-6号生物池NH3_2',
       '6-6号生物池厌氧段MLSS', '6-6号生物池好氧段MLSS', '6-6号生

In [ ]:
for tankid in [str(i+1) for i in range(7)]:
    data=pd.read_csv('./Tank'+tankid+'/data_new.csv')
    print(data.columns)
    n,m=data.shape

    ind=['6-'+tankid+'好氧段曝气量1', '6-'+tankid+'好氧段曝气量2', '6-'+tankid+'好氧段曝气量3', '6-'+tankid+'好氧段曝气量4']
    ind_lag=['6-'+tankid+'生物池DO1', '6-'+tankid+'生物池DO2', '6-'+tankid+'生物池DO3', '6-'+tankid+'生物池DO4']
    rc = {'font.sans-serif': 'SimHei',
        'axes.unicode_minus': False}
    # lag
    lags=[1,30,60,90,120]

    re=['pearson','kendall','spearman']

    for lag in lags:
        for r in re:
            temdata=data.copy()
            temdata[ind_lag].iloc[0:n-lag,:]=temdata[ind_lag].iloc[lag:n,:]
            temdata=temdata.iloc[:n-lag,:]
        
            mat=temdata.corr(method=r)
            plt.figure(figsize=(50,50))
            plt.title(r+'相关性分析，时滞长度：'+str(lag),fontsize=90)
            sns.set(context='notebook', style='ticks', rc=rc)
            sns.set_context({'figure.figsize':[35, 35]})
            ax=sns.heatmap(data=mat, annot=True, linewidths=0.5, square=True, annot_kws={'fontsize':28},fmt='.4f')
            cbar=ax.collections[0].colorbar
            cbar.ax.tick_params(labelsize=40)
            plt.xticks(fontsize=35)
            plt.yticks(fontsize=35)
            plt.savefig('./Results/相关性/all_tank'+tankid+'/lag'+str(lag)+'/corr_'+r+'.png',bbox_inches='tight',dpi=200)
    

# 回归分析

In [ ]:
import tensorflow as tf
from tensorflow import keras

#归一与反归一方法
def normlization(data):
    ma=np.max(data,axis=0)
    mi=np.min(data,axis=0)
    tdata=data.copy()
    for i in range(tdata.shape[0]):
        tdata[i,:]=(data[i,:]-mi)/(ma-mi)
    return tdata,ma,mi

def re_normlization(tdata,ma,mi):
    data=tdata.copy()
    for i in range(data.shape[0]):
        data[i,:]=mi+tdata[i,:]*(ma-mi)
    return data

#数据归一化
td,ma,mi=normlization(data.iloc[:,2:].values)
print(td.shape)

In [ ]:
# 构建MLP和LSTM模型用于回归分析
class MLP:
    def __init__(self,params):
        self.params=params
        tf.compat.v1.disable_eager_execution()
        self.model=keras.models.Sequential()
    
    def _build_net(self):
        self.model.add(
            keras.layers.Dense(units=self.params['layer_in'])
        )
        for layer in range(self.params['layer_num']):
            self.model.add(
                keras.layers.Dense(units=self.params['layer_'+str(layer)])
            )
        self.model.add(
            keras.layers.Dense(units=self.params['layer_out'])
        )
    
    def train(self,X_train,Y_train,X_test,Y_test):
        #op=keras.gradient_descent_v2.optimizers.Adam(learning_rate=self.params['lr'])
        #self.model.compile(optimizer='adam', loss='mse')
        sgd = keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        self.model.compile(loss='mean_squared_error', optimizer=sgd)
        history=self.model.fit(X_train,Y_train,epochs=self.params['ep'],validation_data=(X_test,Y_test),verbose=0)
        return history
    
def mse(x,y):
    return np.sqrt(np.mean(np.square(x-y)))

def nse(x,y):
    a = np.sum(np.square(x-y))/np.sum(np.square(x-np.mean(y)))
    return 1-a
    
X,Y=td[0:td.shape[0]-1,:],td[1:td.shape[0],:]
Xt,Yt = td[5001:6000],td[5002:6001]


In [ ]:
params_mlp={
    'layer_num':5,
    'lr':0.001,
    'ep':500,
}

for i in range(params_mlp['layer_num']):
    params_mlp['layer_'+str(i)]=50
    
params_mlp['layer_out']=Y.shape[1]
params_mlp['layer_in']=X.shape[1]

model_mlp=MLP(params_mlp)
model_mlp._build_net()
his=model_mlp.train(X,Y,Xt,Yt)

#plt.plot(his.history['loss'],label='Training error')
#plt.plot(his.history['val_loss'],label='Validation error')
#plt.legend()

In [ ]:
Ybar=re_normlization(model_mlp.model.predict(X),ma,mi)
YY=re_normlization(Y,ma,mi)

fig,ax=plt.subplots(int(Y.shape[1]/2),2,figsize=(10,20))
k,j=0,0
for i in range(Y.shape[1]):
    ax[k][j].plot(YY[:,i],label='真实值')
    ax[k][j].plot(Ybar[:,i],label='回归值')
    ax[k][j].legend()
    ax[k][j].set_title(data.columns[i+2])
    if k<int(Y.shape[1]/2)-1:
        ax[k][j].set_xticks([])
    else:
        ax[k][j].set_xlabel('时间步')
    
    j+=1
    if j==2:
        k+=1
        j=0

plt.savefig('./Results/回归/all_tank'+tankid+'/reg.png',bbox_inches='tight',dpi=600,pad_inches=0.0)
pd.DataFrame(YY).to_csv('./Results/回归/all_tank'+tankid+'/true.csv')
pd.DataFrame(Ybar).to_csv('./Results/回归/all_tank'+tankid+'/reg.csv')
pd.DataFrame(Y).to_csv('./Results/回归/all_tank'+tankid+'/true_norm.csv')
pd.DataFrame(model_mlp.model.predict(X)).to_csv('./Results/回归/all_tank'+tankid+'/reg_norm.csv')

results={
    'mse':[mse(Ybar,YY)],
    'nse':[nse(Ybar,YY)],
    'mse_norm':[mse(model_mlp.model.predict(X),Y)],
    'nse_norm':[nse(model_mlp.model.predict(X),Y)],
}
pd.DataFrame(results).to_csv('./Results/回归/all_tank'+tankid+'/results.csv')

# 基础统计与概率分布

In [ ]:
temdata=data.iloc[:,2:]
# 各项最大、最小、平均、Q1、Q2、Q3
results_sat=pd.concat([temdata.max(),temdata.min(),temdata.mean(),
                   temdata.quantile(q=0.25,interpolation='linear'),
                   temdata.quantile(q=0.5,interpolation='linear'),
                   temdata.quantile(q=0.75,interpolation='linear')   
],axis=1)
results_sat.to_csv('./Results/统计/all_tank'+tankid+'/sat.csv')

In [ ]:
state = temdata.columns

plt.figure(figsize=(30,40))
i,j=0,1
for it in state:
    k=i*4+j
    ax=plt.subplot(6,4,k)
    temdata[it].hist()
    if j==1:
        ax.set_ylabel('数量',rotation=90,fontsize=30)
    if i==5:
        ax.set_xlabel('取值',rotation=0,fontsize=30)
    ax.set_title(temdata.columns[k-1],fontsize=30)
    ax.tick_params(labelsize=25)
    j+=1
    if np.mod(j,5)==0:
        i+=1
        j=1
plt.savefig('./Results/统计/all_tank'+tankid+'/DIST.png',bbox_inches='tight',dpi=200)

## 条件概率
### MLSS vs CDO/TN/TP
### 曝气量 vs COD/TN/TP

In [ ]:
import data_filter as df

In [ ]:
sump1=pd.DataFrame(data.iloc[:,2]/data.iloc[:,5]) #c/p
sump2=pd.DataFrame(data.iloc[:,2]+data.iloc[:,6]) #c/n
sump3=pd.DataFrame(data.iloc[:,6]+data.iloc[:,5]) #n/p
data['c_p']=sump1
data['c_n']=sump2
data['n_p']=sump3

rang=df.rang_generate(data.iloc[:,2:])
print(data.iloc[:,2:].columns,rang.keys())

In [ ]:
p=[]
for it in [14,17]:
    plt.figure(figsize=(45,50))
    lid=1
    for iil in [26,27,28]:   #'c/n', 'c/p', 'n/p'
        target = data.columns[it]
        ind = data.columns[iil:iil+1].tolist()
        p.append((target,ind))
        
        #每项数据的挡位编号，0到4
        for l in range(5):
            level=[l]
            #ind=index[0:9]
            ck=[]
            cv,count,indl=df.condition_dis(data,ind,target,level,rang,ck)
            ck=['<'+str(format(rang[target][0],'.0f')),
                '['+str(format(rang[target][0],'.0f'))+','+str(format(rang[target][1],'.0f'))+')',
                '['+str(format(rang[target][1],'.0f'))+','+str(format(rang[target][2],'.0f'))+')',
                '['+str(format(rang[target][2],'.0f'))+','+str(format(rang[target][3],'.0f'))+')',
                '['+str(format(rang[target][3],'.0f'))+','+str(format(rang[target][4],'.0f'))+')',
                '>'+str(format(rang[target][4],'.0f'))]
                
            ax=plt.subplot(5,3,lid+l*3)
            ax.barh(ck,cv)
            if it == 14:
                lab = '好氧段MLSS'
            else:
                lab = '曝气量'
            
            if np.mod(lid+l*3,3)==1:
                ax.set_ylabel('目标值范围：'+'\n'+'['+str(round(rang[target][l],2))+','+str(round(rang[target][l+1],2))+']'+'\n'+lab+'分布取值范围',
                              rotation=90,fontsize=40)
            if l==4:
                ax.set_xlabel('数量',rotation=0,fontsize=50)
            
            if lid+l*3==1:
                ax.set_title(lab+'在不同进水C/N下的取值分布',fontsize=50)
            elif lid+l*3==2:
                ax.set_title(lab+'在不同进水C/P下的取值分布',fontsize=50)
            elif lid+l*3==3:
                ax.set_title(lab+'在不同进水N/P下的取值分布',fontsize=50)
            ax.tick_params(labelsize=40)
        lid+=1
    
    plt.savefig('./Results/统计/all_tank'+tankid+'/con_dist'+lab+'.png',bbox_inches='tight',dpi=300)

# 动力系统相关

In [ ]:
import math
import scipy.linalg as linalg
from scipy.spatial.distance import pdist, cdist

from sklearn.linear_model import LinearRegression

In [ ]:
#matrix decomposition tools
# return U,s,V   A=U*diag(s)*V'
def truncated_svd(A,m=np.inf):
    m=min((m,)+A.shape)
    U,s,Vh=linalg.svd(A)
    tol=max(A.shape)*np.spacing(s[0])
    m=min(m,np.count_nonzero(s>tol))
    return U[:,:m],s[:m],Vh[:m].T

# return U,s   A=U*diag(s)*U'
def truncated_svd_sym(A,m=np.inf):
    m=min(m,A.shape[0])
    S,U=linalg.schur(A)
    s=np.diag(S)
    abs_s=np.absolute(s)
    tol=A.shape[0]*np.spacing(abs_s.max())
    m=min(m,np.count_nonzero(abs_s>tol))
    idx=(-abs_s).argsort()[:m]
    return U[:,idx],s[idx]

# return U,s   A=U*diag(s)*U'
def truncated_svd_psd(A,m=np.inf,eps=0):
    m=min(m,A.shape[0])
    S,U=linalg.schur(A)
    s=np.diag(S)
    tol=A.shape[0]*np.spacing(np.absolute(s).max())
    mi=s.min()
    if mi<0 and -mi>tol:
        tol=-mi
    tol = np.maximum(tol, np.absolute(s).max()*eps)
    m=min(m,np.count_nonzero(s>tol))
    idx=(-s).argsort()[:m]
    return U[:,idx],s[idx]

# return R   A=R*R'
def cholcov(A,m=np.inf):
    U,s=truncated_svd_psd(A,m)
    return U*np.sqrt(s)

# return pinv(R)   A=R*R'
def pinv_cholcov(A,m=np.inf,eps=0):
    U,s=truncated_svd_psd(A,m)
    tol=np.absolute(s).max()*eps
    mm=min(m,np.count_nonzero(s>tol))
    return (U[:,:mm]*(1.0/np.sqrt(s[:mm]))).T

# The new observable is U'*f
def truncated_eig(K, m=np.inf, return_reconstruct_K=False):
    m = min(m, K.shape[0])
    w, Q = linalg.eig(K) # K = Q.dot(w).dot(linalg.inv(Q))
    idx = np.argsort(np.abs(w))[::-1]
    w = w[idx]
    Q = Q[:, idx]
    iQ = linalg.inv(Q)
    K_hat = (Q[:, :m] * w[:m]).dot(iQ[:m, :])
    if not np.allclose(np.real(K_hat), K_hat):
        for n in range(m + 1, K.shape[0] + 1):
            K_hat = (Q[:, :n] * w[:n]).dot(iQ[:n])
            if np.allclose(np.real(K_hat), K_hat):
                break
    K_hat = np.real(K_hat)
    U, s, V = truncated_svd(K_hat)
    #print(linalg.norm(K_hat - (U*s).dot(V.T)))
    K_new = (U.T.dot(V * s)).T
    if return_reconstruct_K:
        return K_new, U, K_hat
    return K_new, U

def gramian_function(X, Y=None):
    if Y is None:
        return np.exp(-(cdist(X, X) / 1.5)**2)
    else:
        return np.exp(-(cdist(X, Y) / 1.5)**2)

In [ ]:
class linear_edmd():
    
    def __init__(self, input_dim, feature_mapping):
        '''
        :input_dim: dimension of data
        :feature mapping: A function from (N * input_dim) array to (N * feature_dim) array
        '''
        self.input_dim = input_dim
        self.feature_mapping = feature_mapping
    
    def whiten(self, X, eps=0, return_whitened_data=False):
        #whitening transformation: (Chi-self.whiten_mean).dot(self.whiten_T.T)
        Chi = self.feature_mapping(X)
        m = np.mean(Chi, axis=0)
        C = (Chi - m).T.dot(Chi - m)/X.shape[0]
        T = pinv_cholcov(C, eps=eps)
        self.whiten_mean = m
        self.whiten_T = T
        self.feature_dim = T.shape[1]
        if return_whitened_data:
            return np.hstack([np.ones([Chi.shape[0], 1]), (Chi - m).dot(T.T)])
    
    def whitened_feature_mapping(self, X):
        return np.hstack([np.ones([X.shape[0], 1]), (self.feature_mapping(X)-self.whiten_mean).dot(self.whiten_T.T)])
    
    def train(self, X, Y, eps=0.):
        N = X.shape[0]
        self.X_train = X.copy()
        self.Y_train = Y.copy()        
        Chi_0 = self.whiten(X, eps=eps, return_whitened_data=True)
        Chi_1 = self.whitened_feature_mapping(Y)
        self.K = Chi_0.T.dot(Chi_1) / N
    
    def prediction(self, x0, T, obs_mapping=None, nontrivial_mode_num=np.inf, return_std=False):
        """
        Predict the mean value of the trajectory of the observable from time 1 to time T with x(0)=x0
        :x0: start point
        :T: trajectory length
        :obs_mapping: The observable function from (N * input_dim) array to (N * ...) array. If None, it is identity mapping
        :nontrivial_mode_num: Number of non-trivial singular functions
        :return_std: If return estimated std
        :return: Trajectory of the observable, or (traj, traj_std)
        """
        N = self.Y_train.shape[0]
        m = min(nontrivial_mode_num + 1, self.K.shape[0])
        K, U = truncated_eig(self.K, m)
        m = K.shape[0]
        
        Chi_0 = self.whitened_feature_mapping(self.X_train).dot(U)
        
        if obs_mapping is None:
            O = self.Y_train
        else:
            O = obs_mapping(self.Y_train)
        if return_std:
            O = np.hstack([O, O ** 2])
        
        G = Chi_0.T.dot(O) / N
        
        f0 = self.whitened_feature_mapping(x0.reshape(1, -1)).dot(U).T

        traj = np.empty([T, O.shape[1]])
        for t in range(T):
            traj[t] = (G.T.dot(np.linalg.matrix_power(K.T, t))).dot(f0).reshape(-1)
        if return_std:
            traj_std = np.sqrt(np.maximum(traj[:, (int)(traj.shape[1]/2):] - traj[:, :(int)(traj.shape[1]/2)] ** 2, 0))
            return traj[:, :(int)(traj.shape[1]/2)], traj_std
        return traj

In [ ]:
#tankid='1'
#data=pd.read_csv('./Tank'+tankid+'/data_new.csv')
#print(data.columns)
#n,m=data.shape

X_train = data.iloc[0:data.shape[0]-1,:]
Y_train = data.iloc[1:data.shape[0],:]

In [ ]:
#Perform linear_mmd_dmd and plot singular values
feature_dim = 3000
feature_W = np.random.rand(X_train.shape[1], feature_dim) * 2 -1
feature_b = np.random.rand(feature_dim)
def feature_mapping(X):
    return np.exp(-(X.dot(feature_W) + feature_b) ** 2 / 2)

model_mmd_dmd = linear_edmd(input_dim=X_train.shape[1], feature_mapping=feature_mapping)
model_mmd_dmd.train(X_train, Y_train)
s,U=np.linalg.eig(model_mmd_dmd.K)

eig_data=[]
for ei in s:
    eig_data.append([np.real(ei),np.imag(ei)])
eig_data=np.array(eig_data)

font1 = {'weight' : 'normal',
         'size'   : 20,}

plt.figure(figsize=(5,5))
plt.scatter(eig_data[:,0],eig_data[:,1])
plt.axhline(y=0,ls=":",c="grey")#添加水平直线
plt.axvline(x=0,ls=":",c="grey")#添加水平直线
theta = np.linspace(0, 2 * np.pi, 200)
x = np.cos(theta)
y = np.sin(theta)
plt.plot(x, y, color="darkred", linewidth=1,linestyle=':')
plt.xlabel('实部',font1)
plt.ylabel('虚部',font1)
plt.title(tankid+'号池数据Koopman算子特征值',font1)

plt.savefig('./Results/动力系统相关/all_tank'+tankid+'/eigenvalues.png', bbox_inches='tight', dpi=500)   